## Image Augmentation

Cats v Dogs 로 다음처럼 모델링 하고, 학습시켜본다.

4 convolutional layers with 32, 64, 128 and 128 convolutions

train for 100 epochs

In [2]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O ./cats_and_dogs_filtered.zip



--2025-10-02 02:34:01--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.101.207, 142.251.2.207, 142.250.141.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.101.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘./cats_and_dogs_filtered.zip’

./cats_and_dogs_fil 100%[===================>]  65.43M   331MB/s    in 0.2s    

2025-10-02 02:34:01 (331 MB/s) - ‘./cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [6]:
from tensorflow import keras

In [3]:
import os, zipfile

local_zip = './cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./')
zip_ref.close()

In [4]:
base_dir = './cats_and_dogs_filtered'
train_dir = './cats_and_dogs_filtered/train'
validation_dir = './cats_and_dogs_filtered/validation'

In [8]:
IMG_SIZE = 150
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

In [7]:
# 파일로 저장된 이미지를 불러오는 라이브러리

In [10]:
train_ds = keras.utils.image_dataset_from_directory( train_dir ,
                                         image_size= (IMG_SIZE, IMG_SIZE),
                                          batch_size = 20,
                                          shuffle= True,
                                          seed= 40)

Found 2000 files belonging to 2 classes.


In [12]:
validation_ds = keras.utils.image_dataset_from_directory( validation_dir,
                                         image_size = (IMG_SIZE, IMG_SIZE) ,
                                          batch_size = 20,
                                          shuffle= False)

Found 1000 files belonging to 2 classes.


In [13]:
# 클래스 확인
train_ds.class_names

['cats', 'dogs']

In [14]:
# 성능 최적화를 위해서, 배치로 가져오는 데이터를 백그라운드에서 미리 준비하는 코드

In [16]:
import tensorflow as tf

In [19]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(AUTOTUNE)
validation_ds = validation_ds.prefetch(AUTOTUNE)

In [21]:
# 원래의 이미지를 변형시켜서, 데이터를 풍부하게 만든다. => 이미지 데이터 증강
### Image Data Augmentation ###

In [22]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout

In [23]:
data_augmentation = Sequential()
data_augmentation.add( keras.layers.RandomFlip("horizontal") )   # 원본 이미지를, 좌우 대칭으로 바꾸는것
data_augmentation.add( keras.layers.RandomRotation(0.1)  )    # 원본이미지를 10% 회전시키는것.
data_augmentation.add( keras.layers.RandomZoom(0.1) )  # 원본이미지를 앞 뒤로 확대축소하는것.
data_augmentation.add( keras.layers.RandomTranslation( 0.1, 0.1 ) )  # 왼쪽 오른쪽, 위 아래로 10%씩 이동시키는것.
data_augmentation.name = 'data_augmentation'

In [24]:
data_augmentation = Sequential( [ keras.layers.RandomFlip("horizontal") ,
                                  keras.layers.RandomRotation(0.1) ,
                                  keras.layers.RandomZoom(0.1) ,
                                  keras.layers.RandomTranslation( 0.1, 0.1 ) ],name='data_augmentation')


In [47]:
model = Sequential()
model.add( keras.Input(shape= IMG_SHAPE  )  )    # (150,150,3) => 변수로 이미 만들어놨다.

# 이미지 데이터 증강하는 부분
model.add( data_augmentation )

# 피처 스케일링 하는 부분
model.add( keras.layers.Rescaling(1/255.0) )  # 피처스케일링 한다. Rescaling클래스는 1/255.0 으로 사용.

# 이미지의 특징을 잡는 부분
model.add(  Conv2D( filters= 64, kernel_size= (3,3) , activation= keras.activations.relu )  )
model.add( MaxPooling2D( pool_size=(2,2) ) )
model.add(  Conv2D( filters= 64, kernel_size= (3,3) , activation= keras.activations.relu )  )
model.add( MaxPooling2D( pool_size=(2,2) ) )
model.add(  Conv2D( filters= 128, kernel_size= (3,3) , activation= keras.activations.relu )  )
model.add( MaxPooling2D( pool_size=(2,2) ) )

# 분류하는 부분 FC (ANN)
model.add( Flatten()  )
model.add( Dense(units= 128, activation= keras.activations.relu ))
model.add( Dropout(0.2) )
model.add( Dense(units= 1, activation=keras.activations.sigmoid) )

In [48]:
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ data_augmentation (Sequential)  │ (None, 150, 150, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling_4 (Rescaling)         │ (None, 150, 150, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 148, 148, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 74, 74, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 72, 72, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 128)            │     4,735,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,847,809 (18.49 MB)

 Trainable params: 4,847,809 (18.49 MB)

 Non-trainable params: 0 (0.00 B)

In [49]:
# 컴파일 => 0 이냐 1이냐 분류의 문제 => 0 cats, 1 dogs

In [50]:
model.compile( optimizer= keras.optimizers.RMSprop() ,
               loss = keras.losses.BinaryCrossentropy(),
               metrics= [keras.metrics.BinaryAccuracy() ])

In [51]:
# 얼리스타핑, 모델체크포인트 사용해서, 오버피팅 되기정의 최상의 모델을 찾자

In [52]:
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [53]:
check_point = keras.callbacks.ModelCheckpoint( 'catsdogs_model.keras',
                                              monitor= 'val_loss',
                                               save_best_only= True,
                                               verbose = 1)

In [54]:
# 학습

In [55]:
# X_train, y_train 은 train_ds 에 모두 들어있다.
# 왜냐면, train_ds는 폴더의 파일들을 다 읽어서 가지고 있기때문
model.fit(train_ds, epochs= 20, validation_data= validation_ds, callbacks=[early_stopping,check_point])

Epoch 1/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - binary_accuracy: 0.5112 - loss: 0.7658
Epoch 1: val_loss improved from inf to 0.69275, saving model to catsdogs_model.keras
100/100 ━━━━━━━━━━━━━━━━━━━━ 8s 43ms/step - binary_accuracy: 0.5111 - loss: 0.7653 - val_binary_accuracy: 0.4990 - val_loss: 0.6928
Epoch 2/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - binary_accuracy: 0.5301 - loss: 0.6947
Epoch 2: val_loss improved from 0.69275 to 0.68921, saving model to catsdogs_model.keras
100/100 ━━━━━━━━━━━━━━━━━━━━ 5s 42ms/step - binary_accuracy: 0.5299 - loss: 0.6947 - val_binary_accuracy: 0.5600 - val_loss: 0.6892
Epoch 3/20
 99/100 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - binary_accuracy: 0.5682 - loss: 0.6901
Epoch 3: val_loss improved from 0.68921 to 0.67375, saving model to catsdogs_model.keras
100/100 ━━━━━━━━━━━━━━━━━━━━ 4s 43ms/step - binary_accuracy: 0.5679 - loss: 0.6901 - val_binary_accuracy: 0.5520 - val_loss: 0.6738
Epoch 4/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - bina

In [57]:
model.evaluate(validation_ds)

50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - binary_accuracy: 0.7974 - loss: 0.4942


[0.5961864590644836, 0.699999988079071]